In [1]:
import os
os.sys.path.append(os.path.dirname(os.path.abspath('.')))

# 数据准备

In [2]:
from dataset.dataset import load_mnist

train_data,test_data=load_mnist(batch_size=64)

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(60000, 784) (60000,)
(10000, 784) (10000,)


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# 网络结构设计
LeNet-5的网络结构如下表所示：

|kernel|n_kernel|padding|stride|
|-|:-:|:-:|:-:|
|Conv 5*5|5|2|1|
|MaxPool 2*2|-|0|2|
|Conv 5*5|16|0|1|
|MaxPool 2*2|-|0|2|
|Conv 1*1|120|0|1|
|FC 84|-|-|-|
|Output 10|-|-|-|

In [3]:
import tensorflow as tf

unit_I = train_data.n_features    # 输入单元数，等于特征数

n_filters = [5, 16, 120]    # 卷积核数量
conv_sizes = [(5, 5), (5, 5), (1, 1)]    # 卷积核尺寸

pool_size = (2, 2)    # 池化核尺寸
strides = (2, 2)    # 核移动的步长

FC_size = 84    # 全连接层单元数

unit_O = 10    # 输出单元数，类别数

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 搭建网络

In [4]:
# 输入必须是可由用户指定的，所以设为placeholder
X = tf.placeholder(tf.float32, [None, unit_I])  # 数据的样本数不指定，只指定特征数
Y = tf.placeholder(tf.int64, [None])    # 目标值为列向量，int64为了兼容
# 转为图片格式送入模型，(n_samples,width,height,depth)
X_img = tf.reshape(X, [-1, 28, 28, 1])
training = tf.placeholder_with_default(False, shape=[], name='training')

# 网络结构图
with tf.name_scope('LeNet-5'):
    C1 = tf.layers.conv2d(X_img, filters=n_filters[0],
                          kernel_size=conv_sizes[0], padding='same',
                          activation=tf.nn.tanh, name='C1')
    S2 = tf.layers.max_pooling2d(C1, pool_size=pool_size,
                                 strides=strides, name='S2')
    C3 = tf.layers.conv2d(S2, filters=n_filters[1],
                          kernel_size=conv_sizes[1],
                          activation=tf.nn.tanh, name='C3')
    S4 = tf.layers.max_pooling2d(C3, pool_size=pool_size,
                                 strides=strides, name='S4')
    C5 = tf.layers.conv2d(S4, filters=n_filters[2],
                          kernel_size=conv_sizes[2],
                          activation=tf.nn.tanh, name='C5')
    FC6 = tf.layers.dense(tf.layers.flatten(
        C5), FC_size, activation=tf.nn.tanh)
    # 最后一层直接输出logits，无激活函数
    logits = tf.layers.dense(FC6, unit_O, activation=None)

# 评估图
with tf.name_scope('Eval'):
    # 计算一维向量与onehot向量之间的损失
    loss = tf.losses.sparse_softmax_cross_entropy(labels=Y, logits=logits)
    predict = tf.argmax(logits, 1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), tf.float32))

# 优化图
with tf.name_scope('train_op'):
    lr = 1e-3
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)

init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True    # 按需使用显存

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


# 训练网络

In [5]:
import numpy as np

with tf.Session(config=config) as sess:
    sess.run(init)
    epochs = 20

    batch_cnt = 0
    for epoch in range(epochs):
        for batch_data, batch_labels in train_data.next_batch():
            batch_cnt += 1
            loss_val, acc_val, _ = sess.run(
                [loss, accuracy, train_op],
                feed_dict={
                    X: batch_data,
                    Y: batch_labels,
                    training: True})

            # 每1000batch输出一次信息
            if (batch_cnt+1) % 1000 == 0:
                print('epoch: {}, batch_loss: {}, batch_acc: {}'.format(
                    epoch+1, loss_val, acc_val))

            # 每5000batch做一次验证
            if (batch_cnt+1) % 5000 == 0:
                all_test_acc_val = list()
                for test_batch_data, test_batch_labels in test_data.next_batch():
                    test_acc_val = sess.run(
                        [accuracy],
                        feed_dict={
                            X: test_batch_data,
                            Y: test_batch_labels
                        })
                    all_test_acc_val.append(test_acc_val)
                test_acc = np.mean(all_test_acc_val)
                print('epoch: {}, test_acc: {}'.format(epoch+1, test_acc))

epoch: 2, batch_loss: 0.05612795799970627, batch_acc: 1.0
epoch: 3, batch_loss: 0.02545132488012314, batch_acc: 1.0
epoch: 4, batch_loss: 0.0660727322101593, batch_acc: 0.984375
epoch: 5, batch_loss: 0.025016916915774345, batch_acc: 1.0
epoch: 6, batch_loss: 0.011777980253100395, batch_acc: 1.0
epoch: 6, test_acc: 0.9857683181762695
epoch: 7, batch_loss: 0.003964349161833525, batch_acc: 1.0
epoch: 8, batch_loss: 0.0014940914697945118, batch_acc: 1.0
epoch: 9, batch_loss: 0.0018690538126975298, batch_acc: 1.0
epoch: 10, batch_loss: 0.007561715319752693, batch_acc: 1.0
epoch: 11, batch_loss: 0.003070209175348282, batch_acc: 1.0
epoch: 11, test_acc: 0.984175980091095
epoch: 12, batch_loss: 0.0034181769005954266, batch_acc: 1.0
epoch: 13, batch_loss: 0.03388844430446625, batch_acc: 0.984375
epoch: 14, batch_loss: 0.003671220736578107, batch_acc: 1.0
epoch: 15, batch_loss: 0.007189885713160038, batch_acc: 1.0
epoch: 16, batch_loss: 0.009200932458043098, batch_acc: 1.0
epoch: 16, test_acc: 0